In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
from time import sleep
import random
import re

In [4]:
# generate a list of user agents from stored csv
def user_agent_generator():
    agent_list = pd.read_csv('user_agents.csv')['user_agent']
    return agent_list

# test user agent generator
user_agents = user_agent_generator()
user_agents[:5]

0    Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0)...
1    Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...
2    Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5)...
3    Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...
4    Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...
Name: user_agent, dtype: object

In [7]:
# Function to create a soup object from Coursera's all-course directory

def make_soup(page):
    url = f"https://www.coursera.org/courses?page={page}&index=prod_all_products_term_optimization"
    
    user_agent = random.choice(user_agents)
    
    response = requests.get(url, 
                            headers={'User-Agent': user_agent}, 
                            timeout=10)
    sleep(5)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [19]:
# test with research results page 1
soup = make_soup(1)
# print(soup.prettify())
print(soup.get_text())

Courses | CourseraExploreFor EnterpriseFor StudentsLog InJoin for FreeBrowse>CourseraCoursera CoursesAllGuided ProjectsDegrees & CertificatesShowing 8413 total resultsOther topics to exploreArts and Humanities338 coursesBusiness1095 coursesComputer Science668 coursesData Science425 coursesInformation Technology145 coursesHealth471 coursesMath and Logic70 coursesPersonal Development137 coursesPhysical Science and Engineering413 coursesSocial Sciences401 coursesLanguage Learning150 coursesWhat Coursera Has to Offerlearning programdescriptionGUIDED PROJECTLearn a job-relevant skill that you can use today in under 2 hours through an interactive experience guided by a subject matter expert. Access everything you need right in your browser and complete your project confidently with step-by-step instructions.COURSETake courses from the world's best instructors and universities. Courses include recorded auto-graded and peer-reviewed assignments, video lectures, and community discussion forums.

#### The research results do not appear in the extracted text
- Looks like the table of results is dynamic content?
- Also, moving to other pages doesn't change the URL query. 

In [20]:
# Functions to extract course results data 

course_list = []

def extract_course(soup):
    results = soup.find_all('a', class_ = 'rc-MobileSearchCard')
    for item in results:
        try:
            partner = item.find('spam', class_ = 'partner-name').text.strip()
        except:
            partner = np.nan
        try:
            title = item.find('h2', class_ = 'color-primary-text card-title headline-1-text').text.strip()
        except:
            title = np.nan
        try:
            learning_product = item.find('div', class_ = '_jen3vs _1d8rgfy3').text.strip()
        except:
            learning_product = np.nan
        try:
            rating = item.find('span', class_ = 'ratings-text').text.strip()
        except:
            rating = np.nan
        try:
            rating_count = item.find('span', class_ = 'ratings-count').text.strip()
        except:
            rating_count = np.nan
        try:
            students = item.find('span', class_ = 'enrollment-number').text.strip()
        except:
            students = np.nan
        try:
            difficulty = item.find('span', class_ = 'difficulty').text.strip()
        except:
            difficulty = np.nan
        try:
            link = item.find('a')['href']
        except:
            link = np.nan
                    
            
        course = {
        'partner': partner,
        'title': title,
        'learning_product': learning_product,
        'rating': rating,
        'rating_count': rating_count,
        'students': students,
        'full_description': full_description,
        'difficulty': difficulty,
        'link': link
    }

        course_list.append(course)

    return course_list

In [21]:
# test on search result page 1
extract_course(soup)

[]

### Try Selenium

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [26]:
driver = webdriver.Chrome(executable_path='/Users/nancybui/Downloads/chromedriver')

In [27]:
driver.get('https://www.coursera.org/courses')